In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


import random
import torch
from torch import nn

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [2]:
data = pd.read_csv('preprocessed_data.csv')

In [3]:
data.head()

,Age,BusinessTravel,DistanceFromHome,Education,Gender,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,...,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Attrition
0,0.785714,0.5,0.178571,0.25,0.0,0.00,0.637546,0.111111,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,0.309524,1.0,0.321429,0.00,0.0,0.00,0.167457,0.000000,0.857143,0.333333,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1
2,0.333333,1.0,0.571429,0.75,1.0,0.75,0.964666,0.111111,0.285714,1.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0
3,0.476190,0.0,0.035714,1.00,1.0,0.50,0.385045,0.333333,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
4,0.333333,0.5,0.321429,0.00,1.0,0.00,0.070195,0.444444,0.071429,0.666667,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0


In [4]:
from torch.utils.data import DataLoader, Dataset

In [5]:
class DataSet(Dataset):
    def __init__(self, X, y=None, mode='train'):
        self.X = X
        self.y = y
        self.mode=mode
        
    def __len__(self,):
        return len(self.X)
    
    def __getitem__(self, idx):
        xi = torch.tensor(self.X[idx]).float()
        if self.mode!='test':
            yi = torch.tensor(self.y[idx]).long()
            return xi, yi
        else:
            return xi

In [6]:
X = data.drop('Attrition', 1).to_numpy()
y = data['Attrition'].to_numpy()

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from torchvision.transforms import ToTensor

In [10]:
trainset = DataSet(X_train, y_train, mode='train')
valset = DataSet(X_test, y_test, mode='val')

traingen = DataLoader(trainset, batch_size=32, shuffle=False)
valgen = DataLoader(valset, batch_size=32, shuffle=False)

In [11]:
from collections import OrderedDict

class SimpleNN(nn.Module):
    def __init__(self,):
        super().__init__()
        self.block0=nn.Sequential(
                nn.Linear(41, 100),
                nn.ReLU())
        self.dropout=nn.Dropout(0.2)
        self.block1=nn.Sequential(
                nn.Linear(100, 50),
                nn.BatchNorm1d(50),
                nn.ReLU())
        self.fc=nn.Linear(50, 1)
        
    def forward(self, x):
        x = self.block0(x)
        x = self.dropout(x)
        x = self.block1(x)
        return self.fc(x)

In [12]:
def train(model, criterion, optimizer, scheduler, metric, tgen, vgen, epochs=10, device='cuda:0'):
    cache={'loss':[], 'acc': [], 'v_loss':[], 'v_acc':[], 'epoch':[]}
    for epoch in range(1, epochs+1):
        log = f'::: Epoch {epoch}/{epochs} :::'
        model.train()
        r_loss, r_acc = 0, 0
        for x, y in tgen:
            x, y = x.to(device), y.to(device)
            y_ = model(x)     
            loss = criterion(y_.view(-1), y.float())
            acc = metric(y_.view(-1), y)
            loss.backward()
            
            # Weights update
            optimizer.step()
            optimizer.zero_grad()
            # Weights update
            r_loss += loss.item()
            r_acc += acc.item()

        e_loss, e_acc = r_loss/len(tgen), r_acc/len(tgen)
        cache['loss'].append(e_loss)
        cache['acc'].append(e_acc)
        cache['epoch'].append(epoch)
        template = f' Train ::: Loss: {e_loss:.4f} ::: F1 score: {e_acc:.4f}'
        print(log + template)
        model.eval()
        r_loss, r_acc = 0, 0
        for x, y in vgen:
            x, y = x.to(device), y.to(device)
            y_ = model(x)
            loss, acc = criterion(y_.view(-1), y.float()), metric(y_.view(-1), y)

            r_loss += loss.item()
            r_acc += acc.item()

        e_loss, e_acc = r_loss/len(vgen), r_acc/len(vgen)
        cache['v_loss'].append(e_loss)
        cache['v_acc'].append(e_acc)
        template = f' Validation ::: Loss: {e_loss:.4f} ::: F1 score: {e_acc:.4f}'
        print(log + template)
    return cache, model

In [13]:
def f1_score(pred, target, epsilon=1e-8):
    from sklearn.metrics import f1_score
    pred = (pred > 0).long().cpu().numpy()
    target = target.cpu().numpy()
    return f1_score(pred, target)

In [14]:
set_seed()
model = SimpleNN().to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.9)

In [15]:
history, model = train(model, criterion, optimizer, scheduler, f1_score, traingen, valgen, epochs=30)

::: Epoch 1/30 ::: Train ::: Loss: 0.4782 ::: F1 score: 0.1514
::: Epoch 1/30 ::: Validation ::: Loss: 0.4226 ::: F1 score: 0.0854
::: Epoch 2/30 ::: Train ::: Loss: 0.3771 ::: F1 score: 0.1133
::: Epoch 2/30 ::: Validation ::: Loss: 0.3834 ::: F1 score: 0.2210
::: Epoch 3/30 ::: Train ::: Loss: 0.3463 ::: F1 score: 0.2508
::: Epoch 3/30 ::: Validation ::: Loss: 0.3621 ::: F1 score: 0.3451
::: Epoch 4/30 ::: Train ::: Loss: 0.3219 ::: F1 score: 0.3655
::: Epoch 4/30 ::: Validation ::: Loss: 0.3302 ::: F1 score: 0.4276
::: Epoch 5/30 ::: Train ::: Loss: 0.2889 ::: F1 score: 0.4672
::: Epoch 5/30 ::: Validation ::: Loss: 0.3056 ::: F1 score: 0.4601
::: Epoch 6/30 ::: Train ::: Loss: 0.2648 ::: F1 score: 0.5249
::: Epoch 6/30 ::: Validation ::: Loss: 0.2743 ::: F1 score: 0.5490
::: Epoch 7/30 ::: Train ::: Loss: 0.2379 ::: F1 score: 0.6443
::: Epoch 7/30 ::: Validation ::: Loss: 0.2647 ::: F1 score: 0.6009
::: Epoch 8/30 ::: Train ::: Loss: 0.2099 ::: F1 score: 0.6947
::: Epoch 8/30 ::: V

In [16]:
#torch.save(model, 'fcnn.h5')

In [20]:
def predict(model, testgen, device='cuda:0'):
    model.eval()
    pred = []
    true = []
    for x, y in testgen:
        x, y = x.to(device), y.to(device)
        y_ = model(x)
        true += list(torch.flatten(y.data).cpu().numpy())
        pred += list(torch.flatten(y_.data).cpu().numpy())
    return torch.tensor(pred), torch.tensor(true)

In [21]:
pred, true = predict(model, valgen)

In [22]:
f1_score(pred, true)

0.9750889679715302